In [1]:
from utils import *
from df_utils import *
import openai
import re
import random
import numpy as np
import torch
openai.api_key = open("key.txt", "r").read().strip('\n')

/home/saboa/anaconda3/envs/nlp_final/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [150]:
def get_completion(wall1, model="gpt-4"):
    completion = openai.ChatCompletion.create(
    model=model,
    # max_tokens=125,
    messages=[{"role": "assistant", "content": "for each group describe the relationship in first line and group four words in second single line separated by comma without any further explanation. only write the words in the second line."},
              {"role": "user", "content": "group the following words into 4 groups of related 4: " + wall1}],
                 temperature=0)
    return completion.choices[0].message.content

In [18]:
def get_completion(wall1, model="gpt-4"):
    completion = openai.ChatCompletion.create(
    model=model,
    # max_tokens=125,
    messages=[{"role": "user", "content": "You are an expert contestant on the game show Only Connect and are currently competing in Round 3: “Connecting Wall”. You will be given 16 clues in random order and need to sort them into four groups of connected items. Be careful! The puzzles are designed to include red herrings and to suggest more connections than actually exist, as some clues appear to fit into more than one category. Still, there is only one perfect solution for each wall. Please output your answer as a 4 X 5 markdown table. the last column is the relationship of words in each row. Clues: " + wall1 + " Answer:"}],
                 temperature=0)
    return completion.choices[0].message.content

In [3]:
print(get_completion(model="gpt-4"))

TypeError: get_completion() missing 1 required positional argument: 'wall1'

In [128]:
def examine_wall_gpt(gpt_content, wall1_default):
    GROUP_WORDS = 4
    ITEM_NUM = 16
    reply_content = gpt_content
    lst_words = []
    lst_groups = []
    flag = False

    if '- ' in gpt_content or ' -' in gpt_content:
        flag = True
        reply_content = reply_content.replace('- ', '')
    reply = reply_content.split('\n\n')
    for i in range(4):
        replay_split = reply[i].split('\n')
        if flag:
            lst_words += replay_split[1:]
        else:
            lst_words += replay_split[1].split(', ')
        lst_groups += re.findall(r'(?<=: )(.*)', replay_split[0])*4

    lst_words_default = wall1_default
    correct_groups = 0
    for i in range(GROUP_WORDS, ITEM_NUM+1, GROUP_WORDS):
        for j in range(GROUP_WORDS, ITEM_NUM+1, GROUP_WORDS):
            if sorted(lst_words[i-GROUP_WORDS:i]) == sorted(lst_words_default[j-GROUP_WORDS:j]):
                correct_groups += 1
                # print(lst_words[i-GROUP_WORDS:i])
    return correct_groups, lst_words, lst_groups

In [82]:
def word2index(lst_words, wall1_default):
    dict_bbb = {}
    lst_aaa_new = []
    lst_default = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]
    for i in range(len(wall1_default)):
        dict_bbb[wall1_default[i]] = lst_default[i]
    lst_words_new = lst_words.copy()
    for i in lst_words_new:
        lst_words_new[lst_words_new.index(i)] = dict_bbb[i]
    return lst_words_new, lst_default

In [83]:
# check if there is a '-' in the start of line
# def check_dash(gpt_content):
#     reply = gpt_content.split('\n\n')
#     for i in range(4):
#         replay_split = reply[i].split('\n')
#         if '-' in replay_split[0]:
#             return True
#     return False

In [19]:
def wall_evaluator_gpt(num_default=0, wall_skip=0):
    df_new = DataFrameUtils().preprocess_text()
    wall_info = {}
    num = (num_default * EPISODE_ITEMS) + (wall_skip * WALL_ITEMS)
    wall1_default = df_new[int(num - WALL_ITEMS):int(num)]['Names'].tolist()
    wall_info['names'] = wall1_default
    wall1_shuffled = wall1_default.copy()
    random.shuffle(wall1_shuffled)
    wall1 = " ".join(wall1_shuffled)
    wall1 = wall1.replace(' ', ', ')
    wall1 = wall1.replace('-', ' ')
    print(wall1_default)
    gpt_content = get_completion(wall1).lower()
    # print(gpt_content)
    # correct_groups, lst_words, lst_groups = examine_wall_gpt(gpt_content, wall1_default)
    # # print(correct_groups)
    # # print()
    # # print(lst_words, wall1_default)
    # lst_words_new, lst_default = word2index(lst_words, wall1_default)
    # # print(lst_words_new, lst_default)
    # clf_embeds_metrics = wall_evaluation(label=lst_words_new, label_true=lst_default)
    # wall_info['correct_groups'] = correct_groups
    # wall_info['predicted_relationships'] = lst_groups
    # wall_info['predicted_groups'] = lst_words
    # wall_info['scores'] = clf_embeds_metrics

    # return wall_info
    return gpt_content

In [20]:
aaa = wall_evaluator_gpt(num_default=0, wall_skip=1)

['elastic-band', 'puck', 'squash-ball', 'condom', 'screw', 'grass', 'snout', 'lag', 'santa', 'satan', 'bottom', 'dr-riviera', 'up', 'down', 'charm', 'strange']


In [8]:
aaa

'| puck       | screw      | elastic band | squash ball |\n|------------|------------|--------------|-------------|\n| condom     | snout      | grass        | santa       |\n| strange    | lag        | dr riviera   | charm       |\n| up         | bottom     | down         | satan       |'

In [22]:
aaa

'| up      | down    | bottom  | satan   | opposites / anagrams  |\n|---------|---------|---------|---------|-----------------------|\n| screw   | charm   | strange | dr riviera | types of quarks    |\n| grass   | snout   | puck    | santa   | hidden animals        |\n| lag     | condom  | elastic band | squash ball | rubber-related items |'

In [ ]:
wsreg = '^[ \t]+'

In [52]:
lst_trim = aaa.replace('|', ' ').replace('\n', '').replace('-', '').split('  ')

In [53]:
for i in lst_trim:
    if i == '':
        lst_trim.remove(i)

lst_trim

['up',
 'down',
 'bottom',
 'satan',
 ' opposites / anagrams',
 ' screw',
 ' charm',
 ' strange',
 ' dr riviera',
 ' types of quarks',
 ' grass',
 ' snout',
 ' puck',
 'santa',
 '',
 ' hidden animals',
 '',
 '',
 '',
 '',
 ' lag',
 '',
 '',
 ' condom',
 '',
 'elastic band',
 ' squash ball',
 ' rubberrelated items',
 '']

In [7]:
'scores']['ars']

TypeError: string indices must be integers

In [57]:
'hiho/wareyou'.split('/')[-1]

'wareyou'